In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
 .appName("MovieRecommender") \
 .master("spark://spark-master:7077") \
 .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
 .config("spark.driver.memory", "2g") \
 .config("spark.executor.memory", "3g") \
 .config("spark.driver.host", "recommender-jupyter") \
 .config("spark.driver.bindAddress", "0.0.0.0") \
 .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

:: loading settings :: url = jar:file:/app/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-66200a22-2d0c-49d1-80b9-be72f4f39dde;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.1 in central
	found org.checkerframework#checker-qual;3.41.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.7.1/postgresql-42.7.1.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.7.1!postgresql.jar (133ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.41.0/checker-qual-3.41.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.41.0!checker-qual.jar (64ms)
:: resolution report :: resolve 467ms :: artifacts dl 200ms
	:: modules in use:
	org.checkerfra

In [ ]:
# Load tables
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

ratings.show(5)
movies.show(10)

26/01/29 06:06:28 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/29 06:06:43 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/29 06:06:58 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/29 06:07:13 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/29 06:07:28 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/29 06:07:43 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Prepare data
(training, test) = ratings.randomSplit([0.7, 0.3], seed=42)

# Build ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop"
)

model = als.fit(training)

# Evaluate
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}") # jak interpretować wartość

In [ ]:
# Top 10 recommendations for each user
user_recs = model.recommendForAllUsers(10)
user_recs.show(5, truncate=False)

# jakie filmy ogladal wczesniej a jakie dostal rekomendacje

# Top 10 recommendations for a specific user
user_42_recs = model.recommendForUserSubset(
    spark.createDataFrame([(42,)], ["user_id"]), 10
)

# Join with movie titles
from pyspark.sql.functions import explode

user_42_recs_flat = user_42_recs.select(
    "user_id",
    explode("recommendations").alias("rec")
).select("user_id", "rec.movie_id", "rec.rating")

user_42_recs_flat.join(movies, "movie_id").show(truncate=False)

In [ ]:
from pyspark.sql.functions import explode, col

movie_recs = model.recommendForItemSubset(
    spark.createDataFrame([(1,)], ["movie_id"]), 10
)

# Check the schema
movie_recs.printSchema()

# Explode correctly - it's user_id, not movie_id
movie_recs_flat = movie_recs.select(
    col("movie_id"),
    explode("recommendations").alias("rec")
).select(
    "movie_id",
    col("rec.user_id").alias("user_id"),
    col("rec.rating").alias("score")
)

movie_recs_flat.show()

In [ ]:
!pip install numpy

In [ ]:
from pyspark.ml.feature import Normalizer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
import numpy as np

# Get item factors
item_factors = model.itemFactors

# Get Toy Story's factor vector
toy_story_factor = item_factors.filter(col("id") == 1).select("features").collect()[0][0]

# Calculate cosine similarity
def cosine_sim(v):
    a = np.array(toy_story_factor)
    b = np.array(v)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

cosine_udf = udf(cosine_sim, DoubleType())

# Find similar movies
similar_movies = item_factors \
    .withColumn("similarity", cosine_udf(col("features"))) \
    .filter(col("id") != 1) \
    .orderBy(col("similarity").desc()) \
    .limit(10) \
    .withColumnRenamed("id", "movie_id")

# Join with movie titles
similar_movies.join(movies, "movie_id") \
    .select("movie_id", "title", "similarity") \
    .show(truncate=False)